# Séance 1 — Rappel : Analyse de données avec Pandas (Scraping & préprocessing)

## Objectif

Fournir une méthode pratique et reproductible pour passer d’un **jeu de fichiers mensuels bruts** (NYC Yellow Taxi) à un **jeu de données tabulaire propre, enrichi et prêt pour de l’analyse ou du machine learning**. À l’issue de la séance, vous saurez : charger et concaténer des fichiers parquet/CSV, nettoyer les valeurs aberrantes, parser et exploiter des timestamps, créer des features temporelles et géographiques, agréger par zone/heure, et exporter des livrables réutilisables.

## Description des données

Nous utilisons les **fichiers mensuels Yellow Taxi (NYC)** au format Parquet : `yellow_tripdata_{YYYY}-{MM}.parquet`. Le script fourni permet d’itérer sur une plage temporelle (par défaut `2009-01` → `2025-12`) et de récupérer en plus le fichier de référence des zones `taxi_zone_lookup.csv`.
Colonnes courantes (exemples) :

* `tpep_pickup_datetime`, `tpep_dropoff_datetime` (horodatages)
* `PULocationID`, `DOLocationID` (identifiants de zone)
* `trip_distance`, `passenger_count` (mesures)
* `fare_amount`, `tip_amount`, `total_amount`, `payment_type` (tarification)
  Chaque fichier mensuel contient un grand nombre d’enregistrements : pour une session en salle, privilégier un **échantillon** ou un **mois réduit** plutôt que de charger l’intégralité de la plage en mémoire.

## Récupération des fichiers

Le script de téléchargement utilise les URLs publiques du dataset (`yellow_tripdata_{YYYY}-{MM}.parquet`) et télécharge également `taxi_zone_lookup.csv`. Pour la séance, les options recommandées :

* utiliser un **échantillon** (ex. `yellow_tripdata_sample.csv` ou un seul mois), ou
* télécharger 1–3 mois représentatifs (p. ex. janvier / juillet / décembre d’une année).
  Le script crée les dossiers nécessaires et convertit le lookup CSV en parquet pour accélérer les lectures ultérieures.

## Tâches & transformations avec Pandas

Les opérations couvertes et démontrées dans le TP :

1. **Chargement & inspection**

   * `pd.read_parquet` / `pd.read_csv` ; `head()`, `shape`, `dtypes`, comptage NaN.

2. **Concaténation incrémentale**

   * lire fichier par fichier et concaténer (ou streamer / échantillonner) pour éviter d’épuiser la mémoire.

3. **Nettoyage basique**

   * suppression des lignes impossibles ou aberrantes : `trip_distance <= 0`, `fare_amount <= 0`, `trip_duration <= 0`.
   * normalisation des textes/IDs si nécessaire, gestion des valeurs manquantes (`fillna` / `dropna`).

4. **Parsing et gestion des dates**

   * conversion en `datetime` (`pd.to_datetime`), calcul de la durée (`dropoff - pickup`), détection d’erreurs de parsing.
   * extraction de composantes temporelles : `pickup_hour`, `pickup_day`, `pickup_weekday`, `pickup_month`, `is_night`.

5. **Feature engineering tabulaire**

   * `fare_per_km = fare_amount / trip_distance` (avec gestion des divisions par zéro).
   * `tip_ratio = tip_amount / fare_amount`.
   * indicateurs (ex. `is_long_trip`, `is_shared_zone`) et colonnes dérivées (`n_words` style analogies si texte présent).

6. **Jointures / merges**

   * joindre `taxi_zone_lookup` sur `PULocationID`/`DOLocationID` pour obtenir `pickup_borough`, `pickup_zone` et enrichir l’analyse géographique.

7. **Aggregations & groupby**

   * agrégations par zone / heure / jour : `count`, `mean(trip_distance)`, `median(fare_amount)`, `mean(tip_ratio)`, `max(trip_duration)`.
   * création de tables résumées utiles pour visualisations et features d’agrégation.

8. **Opérations ligne-à-ligne**

   * usage ponctuel de `apply` pour transformations personnalisées (avec mise en garde sur la performance : préférer les opérations vectorisées quand c’est possible).

9. **Export**

   * export final en CSV/Parquet (`cleaned_trips.csv` / `cleaned_trips.parquet`) prêt pour modélisation ou ingestion dans un pipeline ML.

## Livrables attendus

* `s1_pandas.ipynb` : notebook bien commenté (Markdown + cellules de code) avec étapes reproductibles.
* `cleaned_trips.csv` ou `cleaned_trips.parquet` : dataset nettoyé et réduit aux colonnes pertinentes pour ML.
* `summary_by_zone.csv` : table résumé par zone (nombre de trajets, distance moyenne, tarif moyen, ratio pourboire).
* Optional : notebook ou script d’échantillonnage si traitement complet trop lourd.

## Bonnes pratiques et remarques opérationnelles

* **Échantillonnage** : pour la classe, limiter le volume (100k–500k lignes) afin de conserver interactivité.
* **Mémoire** : lire fichier par fichier, utiliser `dtype` explicites et `usecols` pour réduire la charge mémoire.
* **Robustesse** : toujours vérifier `dtypes` après lecture et utiliser `indicator=True` lors des merges pour diagnostiquer les clés non appariées.
* **Chunking des textes** : si vous ajoutez des colonnes textuelles volumineuses (ex. notes), découper avant export pour embeddings.
* **Reproductibilité** : versionner le script de téléchargement et enregistrer la liste des fichiers sources (hashs ou timestamps).

# 1 -  Script de téléchargement des données

In [1]:
import requests
import pandas as pd 
import os
NYC_TRIPS_URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{}-{}.parquet"
DATSET_FOLDER = 'yellow_tripdata'
TAXI_ZONE_URL = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

def verify_if_file_already_downloaded(file_path: str) -> bool:
    """
        Verify if file already downloaded
        Args:
            file_path (str): File path
        Returns:
            bool: True if file already downloaded, False otherwise
    """
    return os.path.exists(file_path)

def format_url(year: str, month: str) -> str:
    """
        Format URL
        Args:
            year (str): Year
            month (str): Month
        Returns:
            str: Formatted URL
    """
    return NYC_TRIPS_URL.format(year, month)

def generate_month_range(start_month : str = '2009-01', 
                         end_month : str = '2025-12'
                        ) -> list:
    """
        Generate month range
        Args:
            start_month (str): Start month
            end_month (str): End month
        Returns:
            list: Month range
    """
    start_year, start_month = int(start_month[:4]), int(start_month[5:])
    end_year, end_month = int(end_month[:4]), int(end_month[5:])  # Correction ici: end_month[5:] au lieu de end_month[4:]
    month_range = []
    for year in range(start_year, end_year + 1):
        for month in range(start_month if year == start_year else 1, end_month + 1 if year == end_year else 13):
            month_range.append(f"{year}-{month:02d}")
    return month_range
def download_data(url: str, file_path: str) -> None:
    """
        Download data from URL
        Args:
            url (str): URL
            file_path (str): File path
        Returns:
            None
    """
    print(f"Downloading data from {url} to {file_path}")
    response = requests.get(url)
    # Check if request was successful
    if response.status_code == 200:
        print(f"Data downloaded from {url} with status code {response.status_code}")
    elif response.status_code == 403 : 
        print(f"File {url} not found, status code {response.status_code}")
        raise Exception(f"File {url} not found, status code {response.status_code}")
    else:
        print(f"Failed to download data from {url} with status code {response.status_code}")
        raise Exception(f"Failed to download data from {url} with status code {response.status_code}")
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, "wb") as f:
        f.write(response.content)
    print(f"Data downloaded from {url} to {file_path}")
    return file_path

def download_data_for_month(year: str, month: str, download_dir: str = f"./data/{DATSET_FOLDER}") -> str:
    """
        Download data for month
        Args:
            year (str): Year
            month (str): Month
            download_dir (str): Download directory
        Returns:
            str: File path
    """
    url = format_url(year, month)
    file_path = os.path.join(download_dir, f"{year}-{month}.parquet")
    if verify_if_file_already_downloaded(file_path):
        print(f"File {file_path} already downloaded, skipping download")
        return file_path
    return download_data(url, file_path)

def download_data_month_to_month(
                                start_month : str = '2009-01', 
                                end_month : str = '2025-12', 
                                download_dir: str = f"./data/{DATSET_FOLDER}"
) -> None:
    """
        Download data month to month
        Args:
            start_month (str): Start month
            end_month (str): End month
            download_dir (str): Download directory
        Returns:
            None
    """
    print(f"Downloading data from {start_month} to {end_month} to {download_dir}")
    month_range = generate_month_range(start_month, end_month)
    for month in month_range:
        year, month_num = month.split('-')
        try:
            download_data_for_month(year, month_num, download_dir)
            print(f"Successfully downloaded data for {year}-{month_num} to {download_dir}")
        except Exception as e:
            print(f"Failed to download data for {year}-{month_num}, error: {e}")
    print(f"Downloaded data from {start_month} to {end_month} to {download_dir}")
    
def download_taxi_zones() -> None:
    """
    Télécharge le fichier des zones de taxi, le convertit en parquet et supprime le CSV.
    """
    url = TAXI_ZONE_URL
    csv_path = "./data/cleaned/taxi_zones.csv"
    parquet_path = "./data/cleaned/taxi_zones.parquet"
    
    # Création du dossier si nécessaire
    os.makedirs("./data/cleaned", exist_ok=True)
    
    # Téléchargement du fichier
    print(f"Téléchargement des zones de taxi depuis {url}")
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Échec du téléchargement des zones de taxi. Code: {response.status_code}")
        return
        
    # Sauvegarde du CSV
    with open(csv_path, "wb") as f:
        f.write(response.content)
    print(f"Fichier CSV sauvegardé: {csv_path}")
    
    # Conversion en parquet
    try:
        df = pd.read_csv(csv_path)
        df.to_parquet(parquet_path)
        print(f"Fichier converti en parquet: {parquet_path}")
        
        # Suppression du CSV
        os.remove(csv_path)
        print("Fichier CSV supprimé")
    except Exception as e:
        print(f"Erreur lors de la conversion: {str(e)}")

In [2]:
download_data_month_to_month(start_month='2025-05', end_month='2026-01')

Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-05.parquet with status code 200
Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-05.parquet to ./data/yellow_tripdata/2025-05.parquet
Successfully downloaded data for 2025-05 to ./data/yellow_tripdata
Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-06.parquet with status code 200
Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-06.parquet to ./data/yellow_tripdata/2025-06.parquet
Successfully downloaded data for 2025-06 to ./data/yellow_tripdata
Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-07.parquet with status code 200
Data downloaded from https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-07.parquet to ./data/yellow_tripdata/2025-07.parquet
Successfully downloaded data for 2025-07 to ./data/yellow_tripdata
Data

## Concaténer tous les fichiers mensuels en un seul DataFrame Pandas

In [3]:
import pandas as pd
import os
DATASET_FOLDER = 'yellow_tripdata'
df_list = []
for file in sorted(os.listdir(f'./data/{DATASET_FOLDER}')):
    if file.endswith('.parquet'):
        df = pd.read_parquet(os.path.join(f'./data/{DATASET_FOLDER}', file))
        df_list.append(df)
        print(f"{file}: {df.shape}")
df = pd.concat(df_list, ignore_index=True)
print(f"Combined DataFrame shape: {df.shape}")

2025-05.parquet: (4591845, 20)
2025-06.parquet: (4322960, 20)
2025-07.parquet: (3898963, 20)
2025-08.parquet: (3574091, 20)
2025-09.parquet: (4251015, 20)
2025-10.parquet: (4428699, 20)
2025-11.parquet: (4181444, 20)
Combined DataFrame shape: (29249017, 20)


In [4]:
df.to_parquet('./data/cleaned/combined_yellow_tripdata_2025.parquet')

In [5]:
import pandas as pd
df = pd.read_parquet('./data/cleaned/combined_yellow_tripdata_2025.parquet')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-05-01 00:07:06,2025-05-01 00:24:15,1.0,3.70,1.0,N,140,202,1,18.40,4.25,0.5,4.85,0.00,1.0,29.00,2.5,0.00,0.75
1,2,2025-05-01 00:07:44,2025-05-01 00:14:27,1.0,1.03,1.0,N,234,161,1,8.60,1.00,0.5,4.30,0.00,1.0,18.65,2.5,0.00,0.75
2,2,2025-05-01 00:15:56,2025-05-01 00:23:53,1.0,1.57,1.0,N,161,234,2,10.00,1.00,0.5,0.00,0.00,1.0,15.75,2.5,0.00,0.75
3,2,2025-05-01 00:00:09,2025-05-01 00:25:29,1.0,9.48,1.0,N,138,90,1,40.80,6.00,0.5,11.70,6.94,1.0,71.94,2.5,1.75,0.75
4,2,2025-05-01 00:45:07,2025-05-01 00:52:45,1.0,1.80,1.0,N,90,231,1,10.00,1.00,0.5,1.50,0.00,1.0,17.25,2.5,0.00,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29249012,2,2025-11-30 23:12:44,2025-11-30 23:43:26,NaN,10.62,NaN,None,68,169,0,33.16,0.00,0.5,0.00,6.94,1.0,44.85,NaN,NaN,0.75
29249013,1,2025-11-30 23:10:35,2025-11-30 23:28:24,NaN,6.50,NaN,None,48,116,0,22.17,0.00,0.5,0.00,0.00,1.0,26.92,NaN,NaN,0.75
29249014,2,2025-11-30 23:09:43,2025-11-30 23:36:08,NaN,8.10,NaN,None,145,152,0,-4.75,0.00,0.5,0.00,0.00,1.0,4.06,NaN,NaN,0.75
29249015,1,2025-11-30 23:29:41,2025-11-30 23:47:09,NaN,5.60,NaN,None,116,48,0,21.42,0.00,0.5,0.00,0.00,1.0,26.17,NaN,NaN,0.75


In [6]:
print("Chargé — lignes :", len(df), "colonnes :", df.shape[1])

Chargé — lignes : 29249017 colonnes : 20


In [7]:
download_taxi_zones()

Téléchargement des zones de taxi depuis https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Fichier CSV sauvegardé: ./data/cleaned/taxi_zones.csv
Fichier converti en parquet: ./data/cleaned/taxi_zones.parquet
Fichier CSV supprimé


In [8]:
taxi_zones_df = pd.read_parquet('./data/cleaned/taxi_zones.parquet')
taxi_zones_df

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,None,None


# Inspection du DataFrame combiné

In [9]:
# Inspection du DataFrame combiné
print("=" * 80)
print("INSPECTION GÉNÉRALE DU DATASET")
print("=" * 80)
print(f"Dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes")
print(f"\nTypes de données:\n{df.dtypes}")
print(f"\n{'Colonne':<30} {'Non-Null':<12} {'Null':<12} {'% Manquant':<12}")
print("-" * 80)
for col in df.columns:
    null_count = df[col].isna().sum()
    non_null_count = len(df) - null_count
    pct_missing = (null_count / len(df)) * 100
    print(f"{col:<30} {non_null_count:<12} {null_count:<12} {pct_missing:>10.2f}%")

INSPECTION GÉNÉRALE DU DATASET
Dimensions: 29249017 lignes × 20 colonnes

Types de données:
VendorID                          int32
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int32
DOLocationID                      int32
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Airport_fee                     float64
cbd_congestion_fee              float64
dtype: object

Colonne                        Non-Null     Null         % Manquant  
-----------------------

In [10]:
print(f"\nStatistiques descriptives:\n")
df.describe()


Statistiques descriptives:



,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
count,2.924902e+07,29249017,29249017,2.184208e+07,2.924902e+07,2.184208e+07,2.924902e+07,2.924902e+07,2.924902e+07,2.924902e+07,2.924902e+07,2.924902e+07,2.924902e+07,2.924902e+07,2.924902e+07,2.924902e+07,2.184208e+07,2.184208e+07,2.924902e+07
mean,1.881741e+00,2025-08-15 15:45:00.798229,2025-08-15 16:02:56.593251,1.293792e+00,7.061845e+00,3.428653e+00,1.606046e+02,1.604135e+02,9.224384e-01,1.837297e+01,1.126894e+00,4.767489e-01,2.823066e+00,5.190812e-01,9.475324e-01,2.684241e+01,2.163961e+00,1.537101e-01,5.329221e-01
min,1.000000e+00,2008-12-31 23:04:21,2008-12-31 23:32:25,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-1.591300e+03,-1.739000e+01,-2.174000e+01,-3.333300e+02,-1.481700e+02,-1.000000e+00,-1.634750e+03,-2.500000e+00,-1.750000e+00,-7.500000e-01
25%,2.000000e+00,2025-06-19 19:04:58,2025-06-19 19:21:03,1.000000e+00,1.070000e+00,1.000000e+00,1.140000e+02,1.070000e+02,0.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.554000e+01,2.500000e+00,0.000000e+00,0.000000e+00
50%,2.000000e+00,2025-08-16 14:35:49,2025-08-16 14:54:49,1.000000e+00,1.910000e+00,1.000000e+00,1.610000e+02,1.620000e+02,1.000000e+00,1.400000e+01,0.000000e+00,5.000000e-01,2.000000e+00,0.000000e+00,1.000000e+00,2.140000e+01,2.500000e+00,0.000000e+00,7.500000e-01
75%,2.000000e+00,2025-10-10 12:01:12,2025-10-10 12:22:00,1.000000e+00,3.900000e+00,1.000000e+00,2.310000e+02,2.330000e+02,1.000000e+00,2.313000e+01,2.500000e+00,5.000000e-01,3.990000e+00,0.000000e+00,1.000000e+00,3.099000e+01,2.500000e+00,0.000000e+00,7.500000e-01
max,7.000000e+00,2025-11-30 23:59:59,2025-12-01 21:41:00,9.000000e+00,3.979944e+05,9.900000e+01,2.650000e+02,2.650000e+02,5.000000e+00,3.254780e+05,1.336000e+02,5.243380e+03,9.609400e+02,7.160500e+02,2.500000e+00,3.255285e+05,2.500000e+00,6.750000e+00,1.750000e+00
std,7.380168e-01,NaN,NaN,7.209231e-01,6.612910e+02,1.489045e+01,6.644989e+01,7.070754e+01,7.592714e-01,8.719872e+01,1.810555e+00,9.794516e-01,4.027244e+00,2.162124e+00,2.894905e-01,8.827560e+01,9.745819e-01,5.448126e-01,3.585713e-01


In [11]:
print(f"\nMémoire utilisée: {df.memory_usage(deep=True).sum() / 1e9:.2f} GB")


Mémoire utilisée: 5.36 GB


# Parsing dates, durée et nettoyage basique

In [12]:
import numpy as np
# Parsing robust des dates
df["tpep_pickup_datetime"]  = pd.to_datetime(df["tpep_pickup_datetime"], errors="coerce")
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"], errors="coerce")

# Calcul durée en minutes
df["trip_duration_min"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60

# Filtrage basique des valeurs manifestement invalides
before = len(df)
df = df[
    (df["trip_distance"] > 0) &
    (df["fare_amount"] > 0) &
    (df["trip_duration_min"] > 0)
].copy()
after = len(df)
print(f"Lignes avant nettoyage : {before} -> après filtres basiques : {after} (supprimées : {before-after})")

# Création de features temporelles
df["pickup_hour"]    = df["tpep_pickup_datetime"].dt.hour
df["pickup_weekday"] = df["tpep_pickup_datetime"].dt.weekday
df["pickup_month"]   = df["tpep_pickup_datetime"].dt.month
df["is_night"]       = df["pickup_hour"].isin([22,23,0,1,2,3,4,5])

# Ratios et indicateurs (avec gestion divisions par zéro)
df["tip_ratio"] = df["tip_amount"] / df["fare_amount"].replace(0, np.nan)
df["fare_per_km"] = df["fare_amount"] / df["trip_distance"].replace(0, np.nan)

# Aperçu des nouvelles colonnes
display(df[["trip_duration_min","trip_distance","fare_amount","tip_ratio","fare_per_km"]].describe().T)


Lignes avant nettoyage : 29249017 -> après filtres basiques : 26063953 (supprimées : 3185064)


,count,mean,std,min,25%,50%,75%,max
trip_duration_min,26063953.0,18.262628,27.213835,1.666667e-02,8.583333,14.066667,22.400000,11295.833333
trip_distance,26063953.0,6.768375,625.988570,1.000000e-02,1.110000,1.950000,3.940000,397994.370000
fare_amount,26063953.0,20.408228,91.691273,1.000000e-02,10.000000,14.900000,24.000000,325478.050000
tip_ratio,26063953.0,0.174742,5.210268,0.000000e+00,0.000000,0.208183,0.286022,20000.000000
fare_per_km,26063953.0,19.396112,208.847990,4.267016e-07,5.443787,7.169811,9.347826,64700.000000


In [13]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,congestion_surcharge,Airport_fee,cbd_congestion_fee,trip_duration_min,pickup_hour,pickup_weekday,pickup_month,is_night,tip_ratio,fare_per_km
0,1,2025-05-01 00:07:06,2025-05-01 00:24:15,1.0,3.70,1.0,N,140,202,1,...,2.5,0.00,0.75,17.150000,0,3,5,True,0.263587,4.972973
1,2,2025-05-01 00:07:44,2025-05-01 00:14:27,1.0,1.03,1.0,N,234,161,1,...,2.5,0.00,0.75,6.716667,0,3,5,True,0.500000,8.349515
2,2,2025-05-01 00:15:56,2025-05-01 00:23:53,1.0,1.57,1.0,N,161,234,2,...,2.5,0.00,0.75,7.950000,0,3,5,True,0.000000,6.369427
3,2,2025-05-01 00:00:09,2025-05-01 00:25:29,1.0,9.48,1.0,N,138,90,1,...,2.5,1.75,0.75,25.333333,0,3,5,True,0.286765,4.303797
4,2,2025-05-01 00:45:07,2025-05-01 00:52:45,1.0,1.80,1.0,N,90,231,1,...,2.5,0.00,0.75,7.633333,0,3,5,True,0.150000,5.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29249010,2,2025-11-30 23:23:56,2025-11-30 23:31:37,NaN,1.28,NaN,None,161,50,0,...,NaN,NaN,0.75,7.683333,23,6,11,True,0.000000,7.539062
29249012,2,2025-11-30 23:12:44,2025-11-30 23:43:26,NaN,10.62,NaN,None,68,169,0,...,NaN,NaN,0.75,30.700000,23,6,11,True,0.000000,3.122411
29249013,1,2025-11-30 23:10:35,2025-11-30 23:28:24,NaN,6.50,NaN,None,48,116,0,...,NaN,NaN,0.75,17.816667,23,6,11,True,0.000000,3.410769
29249015,1,2025-11-30 23:29:41,2025-11-30 23:47:09,NaN,5.60,NaN,None,116,48,0,...,NaN,NaN,0.75,17.466667,23,6,11,True,0.000000,3.825000


# Jointure zones & agrégations rapides

In [14]:
# Charger taxi_zone_lookup (parquet si converti, sinon CSV)

# Merge (left_on PULocationID -> right_on LocationID)
df = df.merge(taxi_zones_df, how="left", left_on="PULocationID", right_on="LocationID", suffixes=("", "_zone"))

# Normaliser noms colonnes utiles après merge
df = df.rename(columns={"Borough":"pickup_borough", "Zone":"pickup_zone"})

# Agrégation par zone de prise en charge
agg_zone = (
    df.groupby("pickup_zone")
      .agg(
          n_trips = ("trip_distance", "count"),
          avg_distance = ("trip_distance", "mean"),
          median_fare = ("fare_amount", "median"),
          avg_tip_ratio = ("tip_ratio", "mean")
      )
      .reset_index()
      .sort_values("n_trips", ascending=False)
)

In [15]:
agg_zone.head(10)

,pickup_zone,n_trips,avg_distance,median_fare,avg_tip_ratio
122,JFK Airport,1183803,15.425462,70.0,0.140835
233,Upper East Side South,1164001,2.591565,10.7,0.220436
155,Midtown Center,1132535,3.275919,13.5,0.203430
232,Upper East Side North,990073,3.052043,11.4,0.212526
182,Penn Station/Madison Sq West,846482,2.847148,14.9,0.201413
156,Midtown East,826019,3.239722,13.5,0.212081
226,Times Sq/Theatre District,799932,3.490638,14.2,0.189961
132,LaGuardia Airport,741692,9.673857,43.6,0.232517
136,Lincoln Square East,734386,2.914455,12.1,0.208515
164,Murray Hill,702936,3.776263,13.5,0.198386


In [16]:
# Agrégation par heure de la journée
agg_hour = (
    df.groupby("pickup_hour")
      .agg(n_trips=("trip_distance","count"),
           avg_fare=("fare_amount","mean"),
           avg_duration=("trip_duration_min","mean"))
      .reset_index()
)
agg_hour

,pickup_hour,n_trips,avg_fare,avg_duration
0,0,827324,21.249208,15.963473
1,1,547305,19.644057,14.875518
2,2,361174,18.276722,13.981765
3,3,245520,18.811815,14.073312
4,4,196990,22.976275,15.788640
5,5,210761,25.791382,17.921756
6,6,415847,22.551342,18.420979
7,7,748902,20.252363,18.362760
8,8,1006534,19.664930,18.057374
9,9,1094485,19.434577,18.031042


In [19]:

# export des résultats échantillonnés
df.to_parquet("./data/cleaned/cleaned_trips_sample.parquet", index=False)
agg_zone.to_csv("./data/cleaned/summary_by_zone_sample.csv", index=False)

# Exercice à faire pour la prochaine séance — Green Taxis (EDA & KPI)

## Contexte & consignes
Reproduisez l’EDA réalisée sur les Yellow Taxis pour le dataset **Green Taxi** (fichiers `green_tripdata_{YYYY}-{MM}.parquet` + `taxi_zone_lookup`). Objectif : produire un notebook `s1_pandas_green.ipynb` contenant une EDA rapide et un fichier KPI agrégé aux niveaux demandés. Travaillez sur **au moins 1 mois** (ou un échantillon 100k–300k lignes si le mois est trop grand). Livrables :
- `s1_pandas_green.ipynb` (notebook commenté)
- `kpi_hourly.csv` (KPI agrégé par heure)
- `kpi_zone.csv` (KPI agrégé par location/zone)
- README court expliquant hypothèses et colonnes utilisées

> Remarque sur l’identifiant chauffeurs : si le dataset contient un identifiant de chauffeur (ex. `driver_id`, `hack_license`, `medallion`), utilisez-le pour calculer le nombre de chauffeurs en course ; sinon, utilisez une combinaison de colonnes stable (ex. `VendorID + some_identifier`) ou documentez l’absence.

## Métriques demandées (4–5) — définitions & dimensions recommandées
1) **Nombre de courses (`n_trips`)**  
    - Formule : `count(*)`  
    - Dimensions (max 3) : `pickup_hour`, `pickup_zone`, `pickup_borough`  
    - Exemple : `pickup_hour, pickup_zone, n_trips`

2) **Nombre de chauffeurs actifs (`n_drivers`)**  
    - Formule : `nunique(driver_id)` (ou `hack_license` / autre identifiant)  
    - Dimensions : `pickup_hour`, `pickup_zone` (ou `pickup_borough`)  
    - Exemple : `pickup_hour, pickup_zone, n_drivers`

3) **Distance moyenne (`avg_distance_km`)**  
    - Formule : `mean(trip_distance)` (préciser si miles ou km)  
    - Dimensions : `pickup_hour`, `pickup_zone`  
    - Exemple : `pickup_hour, avg_distance_km, n_trips`

4) **Tarif moyen et médian (`avg_fare`, `median_fare`)**  
    - Formule : `mean(fare_amount)`, `median(fare_amount)`  
    - Dimensions : `pickup_hour`, `payment_type` (ou `pickup_zone`)  
    - Exemple : `pickup_hour, avg_fare, median_fare, n_trips`

5) **Ratio moyen de pourboire (`avg_tip_ratio`)** — optionnel  
    - Formule : `mean(tip_amount / fare_amount)` avec `fare_amount > 0`  
    - Dimensions : `pickup_hour`, `pickup_zone`, `payment_type`  
    - Exemple : `pickup_hour, pickup_zone, avg_tip_ratio, n_trips`

> Variante : revenu moyen par chauffeur (`sum(total_amount)/n_drivers`) ou courses par chauffeur (`n_trips / n_drivers`).

## Agrégations recommandées (format)
- **KPIs horaires** (`kpi_hourly.csv`) : `pickup_hour, n_trips, n_drivers, avg_distance_km, avg_fare, median_fare, avg_tip_ratio`
- **KPIs par zone** (`kpi_zone.csv`) : `pickup_zone, pickup_borough, n_trips, n_drivers, avg_distance_km, avg_fare, avg_tip_ratio`
- **KPIs hour × zone** (optionnel) : `pickup_hour, pickup_zone, n_trips, n_drivers, avg_fare, avg_tip_ratio`